In [1]:
import json
import shutil
import os
import re
from nuscenes.nuscenes import NuScenes

In [2]:
datasets_root = '/disk/ml/datasets/'
dst_dir = '/disk/ml/own_datasets/CODA/'

In [3]:
shutil.copy(datasets_root + 'CODA/base-val-1500/README.md', dst_dir)
shutil.copy(datasets_root + 'CODA/base-val-1500/corner_case.json', dst_dir)
shutil.copy(datasets_root + 'CODA/base-val-1500/kitti_indices.json', dst_dir)
shutil.copy(datasets_root + 'CODA/base-val-1500/nuscenes_indices.json', dst_dir)

os.mkdir(dst_dir + 'image')
os.mkdir(dst_dir + 'lidar')

NameError: name 'shutil' is not defined

Load Nuscenes

In [8]:
nusc_trainval = NuScenes(version='v1.0-trainval', dataroot=datasets_root + 'nuScenes', verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 57.913 seconds.
Reverse indexing ...
Done reverse indexing in 5.9 seconds.


Get Nuscenes Tokens for the files

In [10]:
json_cornercases = dst_dir + 'corner_case.json'
json_tokens = dst_dir + 'nuscenes_indices.json'
json_nuscenes = datasets_root + 'nuScenes/v1.0-trainval/sample_data.json'
json_nuscenes_image = dst_dir + 'nuscenes_image.json'
json_nuscenes_lidar = dst_dir + 'nuscenes_lidar.json'

with open(json_cornercases, 'r') as f:
    data_cornercases = json.load(f)

with open(json_tokens, 'r') as f:
    data_tokens = json.load(f)

with open(json_tokens, 'r') as f:
    nuscenes_image = json.load(f)
    
with open(json_tokens, 'r') as f:
    nuscenes_lidar = json.load(f)
    

images = data_cornercases['images']

sensor_nuscenes_image = 'CAM_FRONT'
sensor_nuscenes_lidar = 'LIDAR_TOP'


for image in images:
    file_name = image['file_name']
    
    # Check if part of nuScenes  
    if ('nuscenes_' in file_name):

        # Get token ('nuscenes_033402.jpg': '1a41ba0751d5497ebd32df7c86950671')
        token_nuscenes = data_tokens[file_name]

        # Get nuScenes data
        my_sample = nusc_trainval.get('sample', token_nuscenes)
        cam_front_data = nusc_trainval.get('sample_data', my_sample['data'][sensor_nuscenes_image])
        nuscenes_image[file_name] = cam_front_data['filename']
        lidar_top_data = nusc_trainval.get('sample_data', my_sample['data'][sensor_nuscenes_lidar])
        nuscenes_lidar[file_name] = lidar_top_data['filename']

with open(json_nuscenes_image, 'w') as f:
    json.dump(nuscenes_image, f)

with open(json_nuscenes_lidar, 'w') as f:
    json.dump(nuscenes_lidar, f)

Copy Kitti Image

In [ ]:
json_file= dst_dir + 'kitti_indices.json'

src_dir_kitti_image= datasets_root + 'KITTI/object/data/training/image_2/'
dst_dir_kitti_image= dst_dir + 'image/'

with open(json_file, 'r') as file:
    obj = json.load(file)

    for file_name in os.listdir(src_dir_kitti_image):
        source = src_dir_kitti_image + file_name
        
        for file in obj:
            if file.split('_')[1] == file_name:
                print('found')
                destination = dst_dir_kitti_image + file
                
                shutil.copy(source, destination)
                print('copied', file)

Copy Kitti Lidar

In [ ]:
json_file= dst_dir + 'kitti_indices.json'

src_dir_kitti_lidar= datasets_root + 'KITTI/object/data/training/velodyne/'
dst_dir_kitti_lidar= dst_dir + 'lidar/'

with open(json_file, 'r') as file:
    obj = json.load(file)

    for file_name in os.listdir(src_dir_kitti_lidar):
        source = src_dir_kitti_lidar + file_name
        
        for file in obj:
            if file.split('_')[1].split('.')[0] == file_name.split('.')[0]:
                print('found')
                destination = dst_dir_kitti_lidar + 'kitti_' + file_name
                
                shutil.copy(source, destination)
                print('copied', 'kitti_' + file_name)

Copy Nuscenes Image

In [ ]:
json_file= dst_dir + 'nuscenes_image.json'

src_dir_nuscenes_image= datasets_root + 'nuScenes/samples/CAM_FRONT/'
dst_dir_nuscenes_image= dst_dir + 'image/'

with open(json_file, 'r') as file:
    obj = json.load(file)

for file_name in os.listdir(src_dir_nuscenes_image):
    source = src_dir_nuscenes_image + file_name

    
    for file in obj:
        if obj[file].split('/')[2] == file_name:
            print('found')
            destination = dst_dir_nuscenes_image + file
            
            shutil.copy(source, destination)
            print('copied', file)

Copy Nuscenes Lidar

In [ ]:
json_file= dst_dir + 'nuscenes_lidar.json'

src_dir_nuscenes_lidar= datasets_root + 'nuScenes/samples/LIDAR_TOP/'
dst_dir_nuscenes_lidar= dst_dir + 'lidar/'

with open(json_file, 'r') as file:
    obj = json.load(file)

for file_name in os.listdir(src_dir_nuscenes_lidar):
    source = src_dir_nuscenes_lidar + file_name

    
    for file in obj:
        if obj[file].split('/')[2] == file_name:
            print('found')
            destination = dst_dir_nuscenes_lidar + file.split('.')[0] + '.bin'
            
            shutil.copy(source, destination)
            print('copied', file.split('.')[0] + '.bin')

Copy ONCE Image

In [ ]:
json_cornercases = dst_dir + 'corner_case.json'
src_dir_once_image= datasets_root + 'CODA/base-val-1500/images/'
dst_dir_once_image= dst_dir + 'image/'

with open(json_cornercases, 'r') as f:
    data_cornercases = json.load(f)

images = data_cornercases['images']

for image in images:
    file_name = image['file_name']
    if re.search(r'[0-9]{6}_', file_name):
        print('found')
        shutil.copy(src_dir_once_image + file_name, dst_dir_once_image + file_name)
        print('copied', file_name)

Copy ONCE Lidar

In [ ]:
json_cornercases = dst_dir + 'corner_case.json'
src_dir_once_lidar= datasets_root + 'ONCE/data_root/data/'
dst_dir_once_lidar= dst_dir + 'lidar/'

with open(json_cornercases, 'r') as f:
    data_cornercases = json.load(f)

images = data_cornercases['images']

for image in images:
    file_name = image['file_name'].split('.')[0] + '.bin'
    if re.search(r'[0-9]{6}_', file_name):
        print('found')
        shutil.copy(src_dir_once_lidar + file_name.split('_')[0] + '/lidar_roof/' + file_name.split('_')[1], dst_dir_once_lidar + file_name)
        print('copied', file_name)